<a href="https://colab.research.google.com/github/walter118725/TelecomX_Latam_Walter_Malpartida/blob/main/Copia_de_TelecomX_LATAM_Walter_Malpartida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌 Extracción

In [ ]:
import pandas as pd

# ---------- 1) Carga inicial y DF ----------
src_path = "TelecomX_Data.json"
df_raw = pd.read_json(src_path)
df = pd.json_normalize(df_raw.to_dict(orient="records"))

#🔧 Transformación

In [ ]:
import pandas as pd

# ---------- 2) Limpieza base ----------
# Duplicados por ID
df.drop_duplicates(subset="customerID", inplace=True)

# Limpiar Churn vacío y normalizar a minúsculas
df["Churn"] = df["Churn"].replace("", pd.NA).astype("string").str.strip().str.lower()

# Tipos numéricos
df["account.Charges.Monthly"] = pd.to_numeric(df["account.Charges.Monthly"], errors="coerce")
df["account.Charges.Total"]   = pd.to_numeric(df["account.Charges.Total"], errors="coerce").fillna(0)

# Normalizar strings (solo columnas object/string, preservando customerID)
obj_cols = df.select_dtypes(include=["object", "string"]).columns.tolist()
obj_cols = [c for c in obj_cols if c != "customerID"]

for col in obj_cols:
    df[col] = df[col].astype("string").str.strip().str.lower()

# Unificar categorías "no internet/phone service" -> "no"
service_no_map = {
    "no internet service": "no",
    "no phone service": "no"
}
df.replace(service_no_map, inplace=True)

# Eliminar filas sin etiqueta de churn (opcional pero recomendado para modelado)
df = df.dropna(subset=["Churn"])

# ---------- 3) Feature engineering ----------
# Cuentas_Diarias = gasto mensual / 30
df["Cuentas_Diarias"] = df["account.Charges.Monthly"] / 30

# ---------- 4) Estandarización (Yes/No -> 1/0) ----------
yes_no_cols = [
    "Churn",
    "customer.Partner", "customer.Dependents",
    "phone.PhoneService", "phone.MultipleLines",
    "internet.OnlineSecurity", "internet.OnlineBackup", "internet.DeviceProtection",
    "internet.TechSupport", "internet.StreamingTV", "internet.StreamingMovies",
    "account.PaperlessBilling"
]

for col in yes_no_cols:
    if col in df.columns:
        df[col] = df[col].map({"yes": 1, "no": 0}).astype("Int64")

# ---------- 5) Traducciones ligeras y renombrado ----------
# Genero: traducir a español (mantener como categoría)
if "customer.gender" in df.columns:
    df["customer.gender"] = df["customer.gender"].map({"female": "femenino", "male": "masculino"}).fillna(df["customer.gender"])

rename_map = {
    "customerID": "Cliente_ID",
    "Churn": "Evasion",
    "customer.gender": "Genero",
    "customer.SeniorCitizen": "AdultoMayor",
    "customer.Partner": "TienePareja",
    "customer.Dependents": "Dependientes",
    "customer.tenure": "MesesContrato",
    "phone.PhoneService": "ServicioTelefonico",
    "phone.MultipleLines": "MultiplesLineas",
    "internet.InternetService": "ServicioInternet",
    "internet.OnlineSecurity": "SeguridadOnline",
    "internet.OnlineBackup": "RespaldoOnline",
    "internet.DeviceProtection": "ProteccionDispositivo",
    "internet.TechSupport": "SoporteTecnico",
    "internet.StreamingTV": "TV_Cable",
    "internet.StreamingMovies": "PeliculasStreaming",
    "account.Contract": "TipoContrato",
    "account.PaperlessBilling": "FacturaElectronica",
    "account.PaymentMethod": "MetodoPago",
    "account.Charges.Monthly": "CargosMensuales",
    "account.Charges.Total": "CargosTotales",
    "Cuentas_Diarias": "CargosDiarios"
}
df = df.rename(columns=rename_map)

# ---------- 6) Reordenar columnas (opcional) ----------
col_order = [
    "Cliente_ID", "Evasion",
    "Genero", "AdultoMayor", "TienePareja", "Dependientes", "MesesContrato",
    "ServicioTelefonico", "MultiplesLineas",
    "ServicioInternet", "SeguridadOnline", "RespaldoOnline", "ProteccionDispositivo",
    "SoporteTecnico", "TV_Cable", "PeliculasStreaming",
    "TipoContrato", "FacturaElectronica", "MetodoPago",
    "CargosMensuales", "CargosTotales", "CargosDiarios"
]

final_cols = [c for c in col_order if c in df.columns] + [c for c in df.columns if c not in col_order]
df = df[final_cols]

# ---------- 7) Guardar salidas ----------
df.to_csv("TelecomX_Data_Estandarizado.csv", index=False)
df.to_excel("TelecomX_Data_Estandarizado.xlsx", index=False)

#📊 Carga y análisis

In [ ]:
import pandas as pd

#ANÁLISIS DESCRIPTIVO

# Cargar el dataset ya estandarizado (reemplaza la ruta si tu archivo está en otra ubicación)
df = pd.read_csv("TelecomX_Data_Estandarizado.csv")

# Generar estadísticas descriptivas básicas
desc_stats = df.describe().T  # Transpuesta para mejor lectura

# Agregar la mediana manualmente
desc_stats["median"] = df.median(numeric_only=True)

# Mostrar la tabla
print(desc_stats)

# Si quieres guardarla en un archivo Excel
desc_stats.to_excel("Analisis_Descriptivo.xlsx")

#DISTRIBUCIÓN EVACIÓN

import pandas as pd
import matplotlib.pyplot as plt

# 1) Cargar dataset estandarizado (ajusta la ruta si es necesario)
df = pd.read_csv("TelecomX_Data_Estandarizado.csv")

# 2) Conteos y proporciones de evasión
counts = df["Evasion"].value_counts().sort_index()           # 0, 1
props  = df["Evasion"].value_counts(normalize=True).sort_index()

# 3) Gráfico de barras: proporción de clientes por estado de evasión
plt.figure(figsize=(6, 4))
props.plot(kind="bar")
plt.title("Distribución de Evasión (Churn)")
plt.xlabel("Evasión (0 = No, 1 = Sí)")
plt.ylabel("Proporción de clientes")
plt.xticks(rotation=0)

# Anotar porcentajes sobre cada barra
for i, v in enumerate(props.values):
    plt.text(i, v + 0.01, f"{v*100:.1f}%", ha="center")

plt.tight_layout()
# plt.savefig("grafico_evasion_barras.png", dpi=120)
plt.show()

# 4) Gráfico circular: proporción de clientes con/sin evasión
labels = ["No Evasión", "Evasión"]
plt.figure(figsize=(6, 6))
plt.pie(
    counts.values,
    labels=labels,
    autopct="%1.1f%%",
    startangle=90,
    counterclock=False
)
plt.title("Proporción de Clientes con Evasión")
plt.tight_layout()
# plt.savefig("grafico_evasion_pie.png", dpi=120)
plt.show()


#RECUENTO EVASIÓN POR VARIABLES CATEGÓRICAS

import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# -----------------------------
# 1) Cargar dataset
# -----------------------------
df = pd.read_csv("TelecomX_Data_Estandarizado.csv")

# Asegurar que la columna objetivo exista y sea numérica (0/1)
if "Evasion" not in df.columns:
    raise ValueError("No se encontró la columna 'Evasion' en el dataset.")
df["Evasion"] = pd.to_numeric(df["Evasion"], errors="coerce")

# -----------------------------
# 2) Definir variables categóricas a analizar
#    (ajusta la lista según tus columnas disponibles)
# -----------------------------
categ_cols = [
    "Genero",
    "AdultoMayor",          # 0/1 (se tratará como categórico)
    "TienePareja",          # 0/1
    "Dependientes",         # 0/1
    "ServicioTelefonico",   # 0/1
    "MultiplesLineas",      # 0/1
    "ServicioInternet",
    "SeguridadOnline",      # 0/1
    "RespaldoOnline",       # 0/1
    "ProteccionDispositivo",# 0/1
    "SoporteTecnico",       # 0/1
    "TV_Cable",             # 0/1
    "PeliculasStreaming",   # 0/1
    "TipoContrato",
    "FacturaElectronica",   # 0/1
    "MetodoPago",
]

# Filtrar solo las que existan en el DF
categ_cols = [c for c in categ_cols if c in df.columns]

# -----------------------------
# 3) Función de resumen por variable categórica
# -----------------------------
def resumen_churn_por_categoria(data: pd.DataFrame, col: str) -> pd.DataFrame:
    """
    Retorna una tabla con:
    - total_clientes
    - evasores (suma de Evasion)
    - tasa_evasion (%) = mean(Evasion)*100
    Ordenada por tasa de evasión descendente.
    """
    tmp = (
        data.groupby(col, dropna=False)
            .agg(total_clientes=("Evasion", "count"),
                 evasores=("Evasion", "sum"),
                 tasa_evasion=("Evasion", "mean"))
            .reset_index()
    )
    # pasar a porcentaje
    tmp["tasa_evasion"] = (tmp["tasa_evasion"] * 100).round(2)
    # ordenar por tasa desc
    tmp = tmp.sort_values("tasa_evasion", ascending=False)
    return tmp

# -----------------------------
# 4) Generar resúmenes para todas las categóricas
# -----------------------------
resumenes = {}
for col in categ_cols:
    resumenes[col] = resumen_churn_por_categoria(df, col)

# Mostrar ejemplo en consola (las 5 primeras filas de cada tabla)
for col, tabla in resumenes.items():
    print(f"\n=== {col} ===")
    print(tabla.head())

# -----------------------------
# 5) (Opcional) Exportar a Excel, una hoja por variable
# -----------------------------
out_xlsx = "Churn_por_Categorias.xlsx"
with pd.ExcelWriter(out_xlsx) as writer:
    for col, tabla in resumenes.items():
        # Limitar nombre de hoja a 31 caracteres (regla de Excel)
        sheet = col[:31]
        tabla.to_excel(writer, index=False, sheet_name=sheet)

print(f"\nArchivo Excel generado: {Path(out_xlsx).resolve()}")

# -----------------------------
# 6) (Opcional) Gráficos de barras por variable
#     - Un gráfico por variable (matplotlib puro, sin estilos ni colores fijos)
# -----------------------------
guardar_png = True
out_dir = Path("graficos_churn_categorias")
out_dir.mkdir(exist_ok=True)

for col, tabla in resumenes.items():
    # Usar tasa_evasion como altura de las barras
    x_vals = tabla[col].astype(str).fillna("NaN").tolist()
    y_vals = tabla["tasa_evasion"].values

    plt.figure(figsize=(7, 4.5))
    plt.bar(x_vals, y_vals)
    plt.title(f"Tasa de Evasión por {col}")
    plt.xlabel(col)
    plt.ylabel("Tasa de evasión (%)")
    plt.xticks(rotation=45, ha="right")

    # Etiquetas de porcentaje encima de cada barra
    for i, v in enumerate(y_vals):
        plt.text(i, v + 0.5, f"{v:.1f}%", ha="center", va="bottom")

    plt.tight_layout()
    if guardar_png:
        fname = out_dir / f"tasa_evasion_por_{col}.png"
        plt.savefig(fname, dpi=120)
    plt.show()

print(f"\nImágenes guardadas en: {out_dir.resolve()}")


#CONTEO EVASIÓN POR VARIABLES NUMÉRICAS

import pandas as pd
import numpy as np

# Carga tu dataset
df = pd.read_csv("TelecomX_Data_Estandarizado.csv")

# Asegurar que Evasion sea numérica 0/1
df["Evasion"] = pd.to_numeric(df["Evasion"], errors="coerce").fillna(0).astype(int)

# ---- 1) Variables numéricas a analizar ----
num_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c != "Evasion"]

# ---- 2) Tabla A: filas = clase (0/1), columnas = variable_estadística ----
agg_a = (
    df.groupby("Evasion")[num_cols]
      .agg(["count", "mean", "median", "std", "min", "max"])
)
# aplanar nombres de columnas: MesesContrato_mean, CargosTotales_std, etc.
agg_a.columns = [f"{col}_{stat}" for col, stat in agg_a.columns]
tabla_por_clase = agg_a.round(3)

print("\n=== Tabla por clase (filas=Evasion) ===")
print(tabla_por_clase.head())

# ---- 3) Tabla B: filas = variable, columnas = (estadística × clase) ----
agg_b = (
    df.groupby("Evasion")[num_cols]
      .agg(["count", "mean", "median", "std", "min", "max"])
      .T                      # filas: (variable, estadística) ; cols: clase
)
# Opcional: renombrar columnas de clase
agg_b.columns = [f"Evasion={c}" for c in agg_b.columns]
# Dejar un índice amable con nombres
agg_b.index = agg_b.index.set_names(["variable", "estadistica"])

tabla_var_vs_clase = agg_b.round(3)

print("\n=== Tabla variable vs clase (filas=variable,estadística; columnas=clase) ===")
print(tabla_var_vs_clase.head(12))

# ---- 4) (Opcional) Exportar a Excel ----
with pd.ExcelWriter("Numericas_por_Evasion.xlsx") as writer:
    tabla_por_clase.to_excel(writer, sheet_name="por_clase")
    tabla_var_vs_clase.to_excel(writer, sheet_name="variable_vs_clase")

print("\nArchivo exportado: Numericas_por_Evasion.xlsx")


#📄Informe final

# 📊 Informe Final — Churn en TelecomX

**Objetivo:** Analizar la evasión de clientes (*churn*) para comprender patrones de comportamiento, identificar factores asociados a la baja y proponer acciones que permitan **reducir la evasión**.

**Dataset:** 7043 clientes de TelecomX con información demográfica, servicios contratados, condiciones de cuenta y cargos.

**Tasa de evasión estimada:** **26.54%**.


## 🧹 Limpieza y Tratamiento de Datos

- Aplanamiento de la estructura JSON a formato tabular.
- Eliminación de duplicados por `Cliente_ID` y normalización de cadenas (minúsculas, recorte de espacios).
- Estandarización de respuestas binarias (`yes/no`→`1/0`) y traducción de campos al español.
- Conversión de cargos a numérico: `CargosMensuales`, `CargosTotales`.
- Creación de `CargosDiarios = CargosMensuales/30` para granularidad temporal.
- Unificación de categorías especiales: `no internet/phone service → no`.


## 🔎 Análisis Exploratorio de Datos (EDA)

### Distribución de Evasión (0=No, 1=Sí)

![Distribución de Evasión](informe_figs/churn_barras.png)


#### Tasa de Evasión por **TipoContrato**

![Tasa de Evasión por TipoContrato](informe_figs/tasa_evasion_TipoContrato.png)

#### Tasa de Evasión por **MetodoPago**

![Tasa de Evasión por MetodoPago](informe_figs/tasa_evasion_MetodoPago.png)

#### Tasa de Evasión por **ServicioInternet**

![Tasa de Evasión por ServicioInternet](informe_figs/tasa_evasion_ServicioInternet.png)

#### Tasa de Evasión por **Genero**

![Tasa de Evasión por Genero](informe_figs/tasa_evasion_Genero.png)

#### Tasa de Evasión por **FacturaElectronica**

![Tasa de Evasión por FacturaElectronica](informe_figs/tasa_evasion_FacturaElectronica.png)


#### MesesContrato

**Histograma superpuesto**
![Hist MesesContrato](informe_figs/hist_MesesContrato.png)

**Boxplot por Evasión**
![Box MesesContrato](informe_figs/box_MesesContrato.png)

#### CargosMensuales

**Histograma superpuesto**
![Hist CargosMensuales](informe_figs/hist_CargosMensuales.png)

**Boxplot por Evasión**
![Box CargosMensuales](informe_figs/box_CargosMensuales.png)

#### CargosTotales

**Histograma superpuesto**
![Hist CargosTotales](informe_figs/hist_CargosTotales.png)

**Boxplot por Evasión**
![Box CargosTotales](informe_figs/box_CargosTotales.png)

#### CargosDiarios

**Histograma superpuesto**
![Hist CargosDiarios](informe_figs/hist_CargosDiarios.png)

**Boxplot por Evasión**
![Box CargosDiarios](informe_figs/box_CargosDiarios.png)


### Correlación con Evasión (variables numéricas)

![Correlación con Evasión](informe_figs/correlaciones_evasion.png)


## ✅ Conclusiones e Insights

- Mayor tasa de evasión observada en **month-to-month**.
- Clientes con **menor antigüedad** presentan mayor evasión (~**39.5%** vs. **13.2%**).
- Poseer **SeguridadOnline** se asocia a menor evasión (**14.6%** vs. **31.3%**).
- Poseer **SoporteTecnico** se asocia a menor evasión (**15.2%** vs. **31.2%**).
- **Cargos mensuales** altos muestran mayor probabilidad de evasión (~**35.2%** vs. **17.9%**).

## 🧭 Recomendaciones

- **Fidelización por contrato:** Incentivar el paso de contratos mensuales a **anuales/bianuales** con beneficios (descuentos, upgrades).
- **Onboarding y retención temprana:** Priorización de clientes con **baja antigüedad** (primeros 3–6 meses) con seguimiento proactivo y promociones.
- **Paquetes de valor (seguridad/soporte):** Promover **Seguridad Online** y **Soporte Técnico** como diferenciadores que incrementan el valor percibido.
- **Alertas por cargos altos:** Detectar clientes con **cargos mensuales** por sobre la mediana y ofrecer **ajustes de plan** o beneficios de retención.
- **Pagos y fricción:** Revisar métodos de pago y **facturación electrónica** para reducir fricción en la experiencia de cobro.
- **Próximo paso (modelado):** Entrenar un modelo de **clasificación** para identificar clientes en riesgo y activar campañas de retención personalizadas.
